In [ ]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd

# Your USPS API Key
usps_api_key = 'YOUR_API_KEY'

# Function to validate address using USPS API
def validate_address(street, city='Alexandria', state='VA'):
    url = 'http://production.shippingapis.com/ShippingAPI.dll'
    xml = f'''
    <AddressValidateRequest USERID="{usps_api_key}">
        <Revision>1</Revision>
        <Address>
            <Address1></Address1>
            <Address2>{street}</Address2>
            <City>{city}</City>
            <State>{state}</State>
            <Zip5></Zip5>
            <Zip4></Zip4>
        </Address>
    </AddressValidateRequest>'''

    params = {
        'API': 'Verify',
        'XML': xml
    }

    response = requests.get(url, params=params)
    return response.text

# Function to parse USPS response and extract validated address
def parse_usps_response(xml_response):
    root = ET.fromstring(xml_response)
    address = root.find('.//Address2')
    if address is not None:
        return address.text
    return None

# Sample DataFrame with addresses
data = {
    'Address': ['123 Main St', '456 Elm St']  # Add your addresses here
}
df = pd.DataFrame(data)

# Validate and add addresses to DataFrame
validated_addresses = []
for address in df['Address']:
    usps_response = validate_address(address)
    validated_address = parse_usps_response(usps_response)
    validated_addresses.append(validated_address)

# Add the validated addresses as a new column
df['ValidatedAddress'] = validated_addresses

print(df)
